# Exploring dense convolution with Toeplitz transform

This notebook uses the fiber-tree emulator to display the behaviour of various 2-D+muliple output channel convolutions for dense data. Because the data is assumed to be dense we use the position-based operators on the premise that for dense data the position and coordinate are the same.

Note: The extension to multiple input channels should be straightforward, but would result in the Toeplitz tensor being rank-5, which is not displayable in the "uncompressed" form.

First, include some libraries

In [ ]:
# Begin - startup boilerplate code

import pkgutil

if 'fibertree_bootstrap' not in [pkg.name for pkg in pkgutil.iter_modules()]:
  !python3 -m pip  install git+https://github.com/Fibertree-project/fibertree-bootstrap --quiet

# End - startup boilerplate code

from fibertree_bootstrap import *

fibertree_bootstrap(style="uncompressed", animation="movie")

## Logging Control

In [ ]:
enable_log = False

def set_params(log):
    global enable_log

    enable_log = (log == 'enable')

def log(*args):
    if enable_log:
        print(*args)

controls = interactive(set_params,
                       log=['disable', 'enable'])


display(controls)

## Convolution Inputs Selection

Using sliders to select the shapes of the weights and input activations

In [ ]:
H = 5
W = 5
R = 2
S = 2
K = 2


tm = TensorMaker("toeplitz")

tm.addTensor("I_HW",
             rank_ids=["H", "W"],
             shape=[H, W],
             density=1,
             interval=5,
             seed=0,
             color="blue")

tm.addTensor("F_KRS",
             rank_ids=["K", "R", "S"],
             shape=[K, R, S],
             density=1,
             interval=5,
             seed=0,
             color="green")

tm.displayControls()

## Create Input Tensors

Given shapes selected above create and display the filter weights (**f**) and input activations (**i**) and a reference output (**o_verify**)

In [ ]:
I_HW = tm.makeTensor("I_HW")
F_KRS = tm.makeTensor("F_KRS")

print("Input activations")
displayTensor(I_HW)
print("Filter weights")
displayTensor(F_KRS)

H = I_HW.getShape("H")
W = I_HW.getShape("W")
K = F_KRS.getShape("K")
R = F_KRS.getShape("R")
S = F_KRS.getShape("S")

P = H - S + 1
Q = W - R + 1

print("")
print(f"Output activations - {P}x{Q}")


## Output Stationary

$$ O_{k,p,q} = I_{p+r, q+s} \times F_{k,r,s} $$

In [ ]:
o = Tensor.makePopulated(name="Output",
                         rank_ids=["K", "P", "Q"],
                         shape=[K, P, Q])

i = I_HW
f = F_KRS
        
displayTensor(o)

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

canvas = createCanvas(i, f, o)

for k in range(K):
    for p in range(P):
        for q in range(Q):
            for r in range(R):
                for s in range(S):
                    h = p + r
                    w = q + s
                    o[k][p][q] += i[h][w] * f[k][r][s]

                    i_window = [(h,w) for h in range(p, p+S) for w in range(q, q+S)]
                    canvas.addActivity(i_window, (), (), spacetime=("W", (k, p, q, r, s)))
                    canvas.addActivity((h,w), (k,r,s), (k,p,q), spacetime=("P", (k, p, q, r, s)))

displayTensor(o)
displayCanvas(canvas)

## Toplitz conversion

$$ T_{p,q,r,s} = I_{p+r, q+s} $$

In [ ]:
t = Tensor.makePopulated(name="T",
                         rank_ids=["R", "S", "P", "Q"],
                         shape=[R, S, P, Q],
                         color="lightblue")


print("Toplitz matrix creation")

canvas = createCanvas(i, t)

for p in range(P):
    for q in range(Q):
        for r in range(R):
            for s in range(S):
                t[r][s][p][q] <<= i[p+r][q+s]
                canvas.addFrame((p+r,q+s), (r,s,p,q))

displayTensor(t)
displayCanvas(canvas)


## Convolution (using Toeplitz matrix)

$$ O_{k,p,q} = T_{p,q,r,s} \times F_{k,r,s} $$

In [ ]:
o = Tensor.makePopulated(name="O",
                         rank_ids=["K", "P", "Q"],
                         shape=[K, P, Q])

displayTensor(o)

print("Toplitz-based convolution")

canvas = createCanvas(t, f, o)

for k in range(K):
    for p in range(P):
        for q in range(Q):
            for r in range(R):
                for s in range(S):
                    o[k][p][q] += t[r][s][p][q] * f[k][r][s]
                    
                    canvas.addFrame((r,s,p,q), (k,r,s), (k,p,q))

displayTensor(o)
displayCanvas(canvas)


## Flatten Filter Weights

$$F'_{k, r*S+s} = F_{k, r, s} $$

In [ ]:
ff = Tensor.makePopulated(name="Filter Weights Flattened",
                          rank_ids=["K", "RS"],
                          shape=[K, R*S],
                          color="green")
    
displayTensor(ff)

for k in range(K):
    for r in range(R):
        for s in range(S):
            rs = r*S + s
            ff[k][rs] <<= f[k][r][s]
            
displayTensor(ff)

## Flatten Toeplitz Tensor

$$ T'_{p \times Q + q, r \times S + s} = T_{p, q, r, s} $$

In [ ]:
tf = Tensor.makePopulated(name="Toeplitz Flattened",
                          rank_ids=["PQ", "RS"],
                          shape=[P*Q, R*S],
                          color="lightblue")


for p in range(P):
    for q in range(Q):
        pq = p*Q + q
        for r in range(R):
            for s in range(S):
                rs = r*S + s
                tf[pq][rs] <<= t[r][s][p][q]
            
displayTensor(tf)

## Flattened Convolution

$$ O'_{k,pq} = T'_{pq, rs} \times F'_{k, rs} $$

In [ ]:
of = Tensor.makePopulated(name="Output Flattened",
                          rank_ids=["K", "PQ"],
                          shape=[K, P*Q])
 
displayTensor(of)

print("Flattened Toplitz-based convolution")

canvas = createCanvas(tf, ff, of)

for k in range(K):
    for pq in range(P*Q):
        for rs in range(R*S):
            of[k][pq] += tf[pq][rs] * ff[k][rs]
            canvas.addFrame((pq,rs), (k,rs), (k,pq))

displayTensor(of)
displayCanvas(canvas)


## Testing area

For running alternative algorithms